In [ ]:
!pip install optuna
!pip install pytorch_lightning

In [ ]:
#from google.colab import drive
import pickle

import pandas as pd
import numpy as np
import random
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn import model_selection

import matplotlib.pyplot as plt

import optuna

from google.colab import drive
from datetime import datetime, timedelta
drive.mount('/content/drive')
pickle_path = '/content/drive/MyDrive/trendyol_bootcamp_capstone/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The notebook was connected to Google Drive and the Movielens dataset was installed to the relevant directory via the Kaggle API, and then the study continued by reading from there. In order to run the model easily, the relevant data can be accessed from the pickle files. Therefore, the following operations have been commented out and can be skipped.

In [ ]:
'''
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d odedgolden/movielens-1m-dataset
!unzip movielens-1m-dataset.zip -d ./drive/MyDrive/HuaweiTask
drive.mount('/content/drive')
'''

"\n!pip install kaggle\n!mkdir ~/.kaggle\n!cp kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n!kaggle datasets download -d odedgolden/movielens-1m-dataset\n!unzip movielens-1m-dataset.zip -d ./drive/MyDrive/HuaweiTask\ndrive.mount('/content/drive')\n"

In [ ]:
def save_pickle(filename, data):
  outfile = open(pickle_path + filename,'wb')
  pickle.dump(data,outfile)
  outfile.close()

In [ ]:
def load_pickle(filename):
  infile = open(pickle_path+filename,'rb')
  loaded_data = pickle.load(infile)
  infile.close()
  return loaded_data

In [ ]:
test_data = load_pickle('test_data.pickle')
train_data = load_pickle('train_data.pickle')
validation_data = load_pickle('validation_data.pickle')

test_data.reset_index(drop=True,inplace=True)
validation_data.reset_index(drop=True,inplace=True)
train_data.reset_index(drop=True,inplace=True)

In [ ]:
def data_preprocess(data_df):
  data_df['timestamp'] = (data_df['t_dat'] - data_df['t_dat'].iloc[0])/pd.to_timedelta('1Min')
  data_df['timestamp'] = data_df['timestamp'].astype(int)
  data_df.drop(columns = ['t_dat','price','sales_channel_id'], inplace = True)
  data_df['rank'] = 1
  data_df.reset_index(drop=True, inplace = True)

  customer_id_dict = {}
  for i,id in enumerate(list(set(data_df.customer_id))):
    customer_id_dict[id] = i

  mapper = pd.DataFrame.from_dict(customer_id_dict, orient='index')
  mapper = mapper.reset_index(drop=False).rename(columns={'index':'customer_id', 0:'customer_int'})
  data_df = data_df.merge(mapper, on='customer_id')
  data_df['customer_id'] = data_df['customer_int']
  data_df.drop(columns = ['customer_int'],inplace = True)
  return data_df

In [ ]:
#for seasonality
train_data_reduced = train_data[train_data['t_dat']>= '2020-08-28']

train_data = data_preprocess(train_data_reduced)
validation_data = data_preprocess(validation_data)
test_data = data_preprocess(test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

In [ ]:
train_data

,customer_id,article_id,timestamp,rank
0,55666,772902001,0,1
1,55666,189616038,0,1
2,55666,791587001,0,1
3,555,923460001,0,1
4,555,921073001,4320,1
...,...,...,...,...
457403,45666,673677023,15840,1
457404,99825,765308002,15840,1
457405,4389,689365050,15840,1
457406,4389,884081001,15840,1


In order to predict the last movie the user watched, the first movies are included in the training set for the last movie test set. 

# **Negative Sampling**

![picture](https://drive.google.com/uc?export=view&id=1ssu4lJc_IbslkJ_hxeqAelYpQEEmrnRt)

Source: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7 



In [ ]:
train_data.head()

,customer_id,article_id,timestamp,rank
0,55666,772902001,0,1
1,55666,189616038,0,1
2,55666,791587001,0,1
3,555,923460001,0,1
4,555,921073001,4320,1


In [ ]:
(train_data[train_data['customer_id'] == 837319]['article_id']).to_list()

[]

In [ ]:
class ItemDataset:
  def __init__(self, ratings, all_articleIds, negative_sample_count):
    self.customers, self.items, self.labels = self.get_dataset(ratings, all_articleIds, negative_sample_count)

  def __len__(self):
    return len(self.customers)

  def __getitem__(self, idx):
    return self.customers[idx], self.items[idx], self.labels[idx]

  def get_dataset(self, ratings, all_articleIds, negative_sample_count):
    customers, items, labels = [], [], []

    unique_customer_list = list(set(train_data['customer_id']))
    unique_movie_list = list(set(train_data['article_id']))

    for customer_id in unique_customer_list:
      customer_interacted_item_list = (train_data[train_data['customer_id'] == 837319]['article_id']).to_list()
      for article_id in customer_interacted_item_list:
        customers.append(customer_id)
        items.append(article_id)
        labels.append(1)

      possible_not_interacted_movies = set(unique_movie_list) - set(customer_interacted_item_list)
      random_list = random.sample(range(0, len(possible_not_interacted_movies)), negative_sample_count)
      for random_movie_id in random_list:
        customers.append(customer_id)
        items.append(random_movie_id)
        labels.append(0)

    return torch.tensor(customers), torch.tensor(items), torch.tensor(labels)



# **Model**

In [ ]:
class CollaborativeFilteringModel(pl.LightningModule):
  def __init__(self, num_customers, num_items, ratings, all_articleIds):
    super().__init__()
    self.counter = 0
    self.embedding_dim = 8
    self.customer_embedded = nn.Embedding(num_embeddings=num_customers, embedding_dim = self.embedding_dim)
    self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim = self.embedding_dim)
    self.fc1 = nn.Linear(in_features = self.embedding_dim * 2 , out_features = self.embedding_dim * 4)
    self.output = nn.Linear(in_features = self.embedding_dim * 4, out_features=1)
    self.ratings = ratings
    self.all_articleIds = all_articleIds
    self.negative_sample_count = 2
    self.lr = 1e-3

  def forward(self, user_input, item_input):
    
    # Pass through embedding layers
    customer_embedded = self.customer_embedded(user_input)
    item_embedded = self.item_embedding(item_input)

    # Concat the two embedding layers
    vector = torch.cat([customer_embedded, item_embedded], dim=-1)

    # Pass through dense layer
    vector = nn.ReLU()(self.fc1(vector))

    # Output layer
    pred = nn.Sigmoid()(self.output(vector))

    return pred
  
  def training_step(self, batch, batch_idx):
    customer_input, item_input, labels = batch
    predicted_labels = self(customer_input, item_input)
    loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float()) #BCELoss is widely used for Binary Classification
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr = self.lr)

  def train_dataloader(self):
    return DataLoader(ItemDataset(self.ratings, self.all_articleIds, self.negative_sample_count),
                      batch_size=2048, num_workers=4)

Let's think about Netflix movie recommendations, we find the movie that interests us among the movies that are out of our area of interest. We can evaluate the performance of the model with a similar logic. Let's say that one of the 100 different movies on Netflix is the next movie we will watch (the movie we reserved for test data), the rest are movies that we do not interact with. If there is a movie that we will watch among the 10 movies that the model has chosen (recommended) for us, we can say that the model has made a successful prediction.

In [ ]:
def evaluate(model):
  # User-movie pairs for testing
  test_user_movie_set = set(zip(test_data['customer_id'], test_data['article_id']))

  # Dict of all movies that are interacted with by each user
  user_interacted_items = train_data.groupby('customer_id')['article_id'].apply(list).to_dict()
  successful_prediction_counter = 0
  predict_list_size = 100

  for (u,i) in test_user_movie_set:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_articleIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), predict_list_size - 1 ))
    test_items = selected_not_interacted + [i]

    with torch.no_grad():
      predicted_labels = np.squeeze(model(torch.tensor([u] * predict_list_size), 
                                          torch.tensor(test_items)).detach().numpy())
    
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    
    if i in top10_items:
      successful_prediction_counter += 1

  eval_score = successful_prediction_counter / len(test_user_movie_set)

  return eval_score

# **Hyperparameter Tuning**

In [ ]:
num_users = train_data['customer_id'].max()+1
num_items = train_data['article_id'].max()+1

all_articleIds = train_data['article_id'].unique()


In [ ]:
model = CollaborativeFilteringModel(num_users, num_items, train_data, all_articleIds)

evaluation_score = evaluate(model)

In [ ]:
def objective(trial):
  model = CollaborativeFilteringModel(num_users, num_items, train_data, all_articleIds, trial)

  trainer = pl.Trainer(max_epochs = 5, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)
  trainer.fit(model)

  evaluation_score = evaluate(model)

  time = datetime.now().strftime("%H:%M:%S")
  save_pickle('model' + time, model)

  return evaluation_score

In [ ]:
study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=1, n_warmup_steps=0, interval_steps=1
    ),
)
study.optimize(objective, n_trials = 10, timeout=1000)
print(study.best_value, study.best_params)